In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import time
import random
import os
import logging
from datetime import datetime
import os
import sys
import requests
from requests.exceptions import Timeout
from requests.exceptions import RequestException
import re

In [2]:
target_categories = [
    "웹 개발", "프론트엔드", "백엔드", "풀스택", "모바일 앱 개발",
    "프로그래밍 언어", "알고리즘 · 자료구조", "데이터베이스",
    "데브옵스 · 인프라", "자격증 (개발 · 프로그래밍)", "개발 도구", "웹 퍼블리싱",
    "VR/AR", "데스크톱 앱 개발", "게임 개발", "데이터 사이언스", "기타 (개발 · 프로그래밍)",
    "컴퓨터 구조", "임베디드 · IoT", "반도체", "로봇공학", "모빌리티", "자격증 (하드웨어)", "기타 (하드웨어)",
    "게임 프로그래밍", "게임 기획", "게임 아트 · 그래픽", "기타 (게임 개발)",
    "데이터 분석", "인공지능", "데이터 시각화", "데이터 수집 · 처리", "자격증 (데이터 사이언스)",
    "기타 (데이터 사이언스)", "보안", "네트워크", "시스템", "클라우드", "블록체인", "자격증 (보안 · 네트워크)", "기타 (보안 · 네트워크)"
]

In [62]:
url = "https://www.inflearn.com/courses?order=recent&page=1&types=ONLINE"

In [63]:
session = requests.session()
response = session.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [64]:
container = soup.find("div",class_="courses_container")
items = container.find_all(class_=['card', 'course', 'course_card_item'])
item = items[0]


In [66]:
len(items)

24

In [51]:
lecture_id = item.get("data-productid")
a = item.a
img = item.img.get('data-src')
url = "https://inflearn.com"+a.get('href')
course_name = a.find('div',class_='course_title').text
instructor = a.find('div',class_='instructor').text

card_back = item.find("div","course_card_back")
summary = card_back.find("p",class_="course_description")
level = card_back.find("div",class_="course_level").text.strip()
categories = card_back.find("div",class_="course_categories").text.strip().split(",")
tags = card_back.find("div",class_="course_skills").text.strip().split(",")
price = item.find("div","price")
price = {
    "original_price" : price.find("span", class_ = "pay_price").text,
    "sale_price" : price.find("del", class_ = "pay_price").text if price.find("del", class_ = "pay_price") else None
}

data_json = {
    "lecture_id" : lecture_id,
    "url" : url,
    "lecture_name" : course_name,
    "summary" : summary,
    "price" : price,
    "instructor" : instructor,
    "categories" : categories,
    "tags" : tags,
    "level" : level,
    "img" : img,
}

print(data_json)

{'lecture_id': '332809', 'url': 'https://inflearn.com/course/arm아키텍처-메모리모델-배리어', 'lecture_name': 'Arm 아키텍처: 메모리 모델과 배리어 [저자직강 3부-3]', 'summary': <p class="course_description">시스템 소프트웨어의 기본 중의 기본,  최신 Arm 아키텍처(Armv8-A, Armv7-A)의 핵심인 '메모리 모델과 배리어'를 "시스템 소프트웨어 개발을 위한 Arm 아키텍처의 구조와 원리"의 저자가 제대로 잡아드립니다!</p>, 'price': {'original_price': '₩23,100', 'sale_price': None}, 'instructor': '김동현 (Austin Kim)', 'categories': ['임베디드 · IoT', ' 반도체'], 'tags': ['ARM Architecture', ' memory-mapped-io', ' barrier'], 'level': '초급', 'img': 'https://cdn.inflearn.com/public/courses/332809/cover/714d216e-a073-4a66-b827-3a3711ed0db9/332809.png'}


In [32]:
target_categories = set([
        "웹 개발", "프론트엔드", "백엔드", "풀스택", "모바일 앱 개발",
        "프로그래밍 언어", "알고리즘 · 자료구조", "데이터베이스",
        "데브옵스 · 인프라", "자격증 (개발 · 프로그래밍)", "개발 도구", "웹 퍼블리싱",
        "VR/AR", "데스크톱 앱 개발", "게임 개발", "데이터 사이언스", "기타 (개발 · 프로그래밍)",
        "컴퓨터 구조", "임베디드 · IoT", "반도체", "로봇공학", "모빌리티", "자격증 (하드웨어)", "기타 (하드웨어)",
        "게임 프로그래밍", "게임 기획", "게임 아트 · 그래픽", "기타 (게임 개발)",
        "데이터 분석", "인공지능", "데이터 시각화", "데이터 수집 · 처리", "자격증 (데이터 사이언스)",
        "기타 (데이터 사이언스)", "보안", "네트워크", "시스템", "클라우드", "블록체인", "자격증 (보안 · 네트워크)", "기타 (보안 · 네트워크)"
    ]
)

headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0",
    "Accept-Language" : "ko,en;q=0.9,en-US;q=0.8"
}

session = requests.session()
p = 1
crashed = set()
list_dic = {}
while (p<=135):
    url = f"https://www.inflearn.com/courses?order=recent&page={p}&types=ONLINE"
    if (p%10 == 0):
        session = requests.session()
    response = session.get(url, headers=headers, timeout=10)
    if response.status_code != 200:
        raise Exception
    soup = BeautifulSoup(response.text, 'html.parser')
    container = soup.find("div",class_="courses_container")
    items = container.find_all(class_=['card', 'course', 'course_card_item'])
    temp = {}
    for item in items:
        card_back = item.find("div","course_card_back")
        categories = card_back.find("div",class_="course_categories").text.strip().split(",")

        is_target = False
        for c in categories:
            if c in target_categories:
                is_target = True
                break
        if not is_target:
            continue
            
        lecture_id = item.get("data-productid")
        if lecture_id in list_dic:
            # 중복
            continue

        a = item.a
        img = item.img.get('data-src')
        url = "https://inflearn.com"+a.get('href')
        course_name = a.find('div',class_='course_title').text
        instructor = a.find('div',class_='instructor').text

        summary = card_back.find("p",class_="course_description").text
        level = card_back.find("div",class_="course_level").text.strip()
        tags = card_back.find("div",class_="course_skills").text.strip().split(",")
        price = item.find("div","price")

        try:
            if (price.find("del")):
                # 세일중
                # print(int(re.sub(r'[₩,]', '', price.find("del").text)))
                # print(int(re.sub(r'[₩,]', '', price.span.text)))
 
                price = {
                    "is_free" : False,
                    "is_sale" : True,
                    "data" : {
                        "original_price" : int(re.sub(r'[₩,]', '', price.find("del").text)),
                        "sale_price" : int(re.sub(r'[₩,]', '', price.span.text))
                    }
                }
            else:
                if (price.text.strip()=="무료"):
                    price = {
                        "is_free" : True,
                        "is_sale" : False,
                        "data" : {
                            "original_price" : 0,
                            "sale_price" : 0
                        }
                    }
                else:
                    price = {
                        "is_free" : False,
                        "is_sale" : False,
                        "data" : {
                            "original_price" : int(re.sub(r'[₩,]', '', price.text)),
                            "sale_price" : int(re.sub(r'[₩,]', '', price.text))
                        }
                    }

            data_json = {
                "lecture_id" : lecture_id,
                "url" : url,
                "lecture_name" : course_name,
                "summary" : summary,
                "price" : price,
                "instructor" : instructor,
                "categories" : categories,
                "tags" : tags,
                "level" : level,
                "img" : img,
            }
            temp[lecture_id] = data_json        
        except Exception as e:
            print(lecture_id)

    # 전송
    list_dic.update(temp)
    print(list_dic.keys())
    p += 1
    break

if not os.path.exists("./list/"):
    os.makedirs("./list/")
with open("./list/list_data.json", "w", encoding="utf-8") as f:
    json.dump(list_dic, f, ensure_ascii=False)
# done message
    


    

dict_keys(['332809', '332927', '332866', '332863', '332843', '332739', '332034', '331907', '332891', '332857', '332815', '331960', '332846', '332773', '332626'])


In [33]:
list_dic["332809"]

{'lecture_id': '332809',
 'url': 'https://inflearn.com/course/arm아키텍처-메모리모델-배리어',
 'lecture_name': 'Arm 아키텍처: 메모리 모델과 배리어 [저자직강 3부-3]',
 'summary': '시스템 소프트웨어의 기본 중의 기본,  최신 Arm 아키텍처(Armv8-A, Armv7-A)의 핵심인 \'메모리 모델과 배리어\'를 "시스템 소프트웨어 개발을 위한 Arm 아키텍처의 구조와 원리"의 저자가 제대로 잡아드립니다!',
 'price': {'is_free': False,
  'is_sale': True,
  'data': {'original_price': 33000, 'sale_price': 23100}},
 'instructor': '김동현 (Austin Kim)',
 'categories': ['임베디드 · IoT', ' 반도체'],
 'tags': ['ARM Architecture', ' memory-mapped-io', ' barrier'],
 'level': '초급',
 'img': 'https://cdn.inflearn.com/public/courses/332809/cover/714d216e-a073-4a66-b827-3a3711ed0db9/332809.png'}

In [27]:
url = f"https://www.inflearn.com/courses?order=recent&page={500}&types=ONLINE"
session = requests.session()
response = session.get(url, headers=headers, timeout=10)
print(response.status_code)
print(response.status_code)

200


In [23]:
for k,v in list_dic.items():
    print(v)

{'lecture_id': '332809', 'url': 'https://inflearn.com/course/arm아키텍처-메모리모델-배리어', 'lecture_name': 'Arm 아키텍처: 메모리 모델과 배리어 [저자직강 3부-3]', 'summary': <p class="course_description">시스템 소프트웨어의 기본 중의 기본,  최신 Arm 아키텍처(Armv8-A, Armv7-A)의 핵심인 '메모리 모델과 배리어'를 "시스템 소프트웨어 개발을 위한 Arm 아키텍처의 구조와 원리"의 저자가 제대로 잡아드립니다!</p>, 'price': {'is_free': False, 'is_sale': True, 'data': {'original_price': 33000, 'sale_price': 23100}}, 'instructor': '김동현 (Austin Kim)', 'categories': ['임베디드 · IoT', ' 반도체'], 'tags': ['ARM Architecture', ' memory-mapped-io', ' barrier'], 'level': '초급', 'img': 'https://cdn.inflearn.com/public/courses/332809/cover/714d216e-a073-4a66-b827-3a3711ed0db9/332809.png'}
{'lecture_id': '331768', 'url': 'https://inflearn.com/course/빼먹기-쉬운-구조', 'lecture_name': '빼먹기 쉬운 구조: 소비자에서 생산자로. w.개발 Process', 'summary': <p class="course_description">더 이상 “남의 주관”에서 살아가는 “나, 자신”이 아닌. “나”라는 사람으로 살아남기! 사회적으로 굳어진 프레임과 거품을 제거하고, 자기 경험과 커리어를 개발과 디자인의 프로세스를 통해 퍼스널 브랜딩하는 Know + How를 익히게 됩니다.</p>, 'price': {'is_free': F